## Define the Agent state

In [1]:
"""
Portfolio Assistant with LangGraph - AI Agents System with Tool Routing
======================================================================

This notebook implements a LangGraph-based AI agents system with:
- Agent Node: Analyzes user input and decides when to use tools
- Tool Node: Contains all MCP tools and executes them automatically
- Conditional Routing: Routes between agent and tools based on AI decisions

The system connects to the MCP server and uses langchain-mcp-adapters for native integration.
"""

from typing import TypedDict, Annotated

from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

# State management
class AgentState(TypedDict):
    """State shared between all nodes in the workflow"""
    messages: Annotated[list[AnyMessage], add_messages]
    original_user_input: str
    execution_plan: AnyMessage


## Define tools
- Tools from MCP server (persistence)
- Custom tools
- Pre-built tools

### MCP tools

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

mcp_client = MultiServerMCPClient({
    "portfolio": {
        "url": "http://localhost:8081/mcp",
        "transport": "streamable_http",
    }
})

# Get tools from MCP server - this returns actual LangChain tools!
print("🔍 Connecting to MCP server and loading tools...")
try:
    available_tools = await mcp_client.get_tools()
    print(f"✅ Successfully loaded {len(available_tools)} LangChain tools from MCP server!")
    
    print("\n📊 Available Portfolio Tools:")
    for i, tool in enumerate(available_tools, 1):
        print(f"  {i:2d}. {tool.name}: {tool.description}")
    
    print(f"\n🎯 All tools are ready for LangGraph workflow!")
    
except Exception as e:
    print(f"❌ Failed to connect to MCP server: {e}")
    print("⚠️  Please ensure the MCP server is running on localhost:8081")
    print("   Command: python start_portfolio_http_server.py")
    available_tools = []


🔍 Connecting to MCP server and loading tools...
✅ Successfully loaded 12 LangChain tools from MCP server!

📊 Available Portfolio Tools:
   1. createTransaction: Create a new transaction in the portfolio.
   2. deleteTransaction: Delete a transaction by ID.
   3. getAllPositions: Get all current positions in the portfolio.
   4. getPortfolioSummary: Get portfolio summary with key metrics.
   5. getPositionByTicker: Get position details for a specific ticker.
   6. getTransaction: Get a transaction by its ID.
   7. getTransactionsByTicker: Get all transactions for a specific ticker.
   8. recalculateAllPositions: Recalculate all positions from transactions.
   9. recalculatePosition: Recalculate position for a specific ticker.
  10. searchTransactions: Search transactions with multiple filters.
  11. updateMarketData: Update market data for a position.
  12. updateTransaction: Update an existing transaction given its transaciton ID.

🎯 All tools are ready for LangGraph workflow!


### Custom tools for market analysis

In [3]:
from langchain_core.tools import tool
import yfinance as yf
from typing import Dict, Any

@tool
def get_stock_information(symbol: str) -> Dict[str, Any]:
    """ Get current stock price and basic information for a symbol (ticker) from a real time data source. Use this tool when you need to get the current price of a stock."""
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        hist = stock.history(period="1d")

        if hist.empty:
            return {"error": f"No data available for this symbol {symbol}"}

        current_price = hist["Close"].iloc[-1]
        return {
            "symbol": symbol,
            "current_price": current_price,
            "previous_close": float(info.get("previousClose", current_price)),
            "volume": info.get('volume', 0),
            "market_cap": info.get('marketCap'),
            "day_change": float(current_price - info.get('previousClose', current_price)),
            "day_change_percent": float(((current_price - info.get('previousClose', current_price)) / info.get('previousClose', current_price)) * 100) if info.get('previousClose') else 0
        }
    except Exception as e:
        return {"error": f"Error fetching stock info for {symbol}: {str(e)}"}


In [4]:
from langchain_core.tools import tool
from typing import Dict, Any

@tool 
def calculate_position_value(symbol: str, quantity: float, avg_cost: float, current_price: float) -> Dict[str, Any]:
    """Calculate the current value and P&L for a stock position. Use this tool when you need to calculate the current value and P&L for a stock position."""
    market_value = quantity * current_price
    cost_basis = quantity * avg_cost
    gain_loss = market_value - cost_basis
    gain_loss_percent = (gain_loss / cost_basis) * 100 if cost_basis > 0 else 0
    
    return {
        "symbol": symbol,
        "market_value": market_value,
        "cost_basis": cost_basis,
        "gain_loss": gain_loss,
        "gain_loss_percent": gain_loss_percent,
        "current_price": current_price,
        "quantity": quantity,
        "avg_cost": avg_cost
    }

In [5]:

@tool
def detect_stock_splits(symbol: str, days_back: int = 720) -> Dict[str, Any]:
    """ Detect stock splits for a symbol within specified days back. Use this tools when you need to detect stock splits for a symbol (i.e. if a stock price is much lower than the user paid price, it could be a split)"""
    try:
        stock = yf.Ticker(symbol)
        splits = stock.splits
        
        if splits.empty:
            return {"symbol": symbol, "splits_found": False, "splits": []}
        
        # Filter splits within the specified period
        from datetime import datetime, timedelta
        import pandas as pd
        cutoff_date = datetime.now() - timedelta(days=days_back)
        # Handle timezone awareness - convert to UTC if splits.index is timezone-aware
        if splits.index.tz is not None:
            cutoff_date = pd.Timestamp(cutoff_date).tz_localize('UTC')
        recent_splits = splits[splits.index >= cutoff_date]
        
        splits_data = []
        for split_date, split_ratio in recent_splits.items():
            splits_data.append({
                "date": split_date.strftime("%Y-%m-%d"),
                "ratio": float(split_ratio),
                "description": f"{int(split_ratio)}:1 split" if split_ratio >= 1 else f"1:{int(1/split_ratio)} reverse split"
            })
        
        return {
            "symbol": symbol,
            "splits_found": len(splits_data) > 0,
            "splits": splits_data,
            "total_splits": len(splits_data)
        }
    except Exception as e:
        return {"error": f"Error detecting splits for {symbol}: {str(e)}"}

In [6]:
@tool
def calculate_position_value_with_splits(
    symbol: str, 
    original_quantity: float, 
    original_avg_cost: float, 
    purchase_date: str = None
) -> Dict[str, Any]:
    """ Calculate position value accounting for stock splits since purchase. Use this tool when you need to calculate the current value and P&L for a stock position, accounting for splits since purchase. """
    try:
        # Get current stock price
        stock_info = get_stock_information(symbol)
        if "error" in stock_info:
            return stock_info
        
        current_price = stock_info["current_price"]
        
        # Check for splits since purchase date
        if purchase_date:
            from datetime import datetime
            purchase_dt = datetime.strptime(purchase_date, "%Y-%m-%d")
            days_since_purchase = (datetime.now() - purchase_dt).days
            
            splits_info = detect_stock_splits(symbol, days_since_purchase)
            
            # Apply split adjustments
            adjusted_quantity = original_quantity
            adjusted_avg_cost = original_avg_cost
            
            if splits_info["splits_found"]:
                cumulative_split_ratio = 1.0
                for split in splits_info["splits"]:
                    split_date = datetime.strptime(split["date"], "%Y-%m-%d")
                    if split_date >= purchase_dt:
                        cumulative_split_ratio *= split["ratio"]
                
                adjusted_quantity = original_quantity * cumulative_split_ratio
                adjusted_avg_cost = original_avg_cost / cumulative_split_ratio
        else:
            adjusted_quantity = original_quantity
            adjusted_avg_cost = original_avg_cost
            splits_info = {"splits_found": False, "splits": []}
        
        # Calculate position metrics
        market_value = adjusted_quantity * current_price
        cost_basis = adjusted_quantity * adjusted_avg_cost
        gain_loss = market_value - cost_basis
        gain_loss_percent = (gain_loss / cost_basis) * 100 if cost_basis > 0 else 0
        
        return {
            "symbol": symbol,
            "original_quantity": original_quantity,
            "original_avg_cost": original_avg_cost,
            "current_quantity": adjusted_quantity,
            "current_avg_cost": adjusted_avg_cost,
            "current_price": current_price,
            "market_value": market_value,
            "cost_basis": cost_basis,
            "gain_loss": gain_loss,
            "gain_loss_percent": gain_loss_percent,
            "splits_applied": splits_info["splits"],
            "split_adjusted": splits_info["splits_found"]
        }
    except Exception as e:
        return {"error": f"Error calculating position for {symbol}: {str(e)}"}

In [7]:
@tool
def detect_fractional_share_offering(symbol: str, user_paid_price: float) -> Dict[str, Any]:
    """ Detect if broker is offering fractional shares based on price discrepancy. Use this tool when you need to detect if broker is offering fractional shares based on price discrepancy (i.e. if the user paid price is much lower than the actual market price, like x100 or x1000 times lower) """
    try:
        # Get actual market price
        stock_info = get_stock_information(symbol)
        if "error" in stock_info:
            return stock_info
        
        actual_price = stock_info["current_price"]
        price_ratio = actual_price / user_paid_price
        
        # Common fractional ratios
        common_ratios = [
            2, 3, 4, 5, 6, 8, 10, 12, 15, 16, 20, 25, 30, 32, 40, 50, 
            60, 64, 75, 80, 100, 120, 125, 150, 160, 200, 250, 300, 
            400, 500, 600, 750, 800, 1000, 1250, 1500, 2000
        ]
        
        # Find closest ratio
        closest_ratio = min(common_ratios, key=lambda x: abs(x - price_ratio))
        
        # Flexible tolerance - either 10% OR if ratio is > 1.5 (clear fractional indicator)
        tolerance_percentage = 0.15 if price_ratio > 1.5 else 0.05  # 15% for likely fractional, 5% for borderline
        is_fractional = (
            abs(price_ratio - closest_ratio) < (closest_ratio * tolerance_percentage) or 
            price_ratio > 1.5  # Any ratio > 1.5 is likely fractional
        )
        
        # If no close match found but ratio is high, try to find a reasonable approximation
        if not is_fractional and price_ratio > 1.5:
            # Round to nearest logical fraction
            if price_ratio < 10:
                closest_ratio = round(price_ratio)
            elif price_ratio < 100:
                closest_ratio = round(price_ratio / 5) * 5  # Round to nearest 5
            elif price_ratio < 1000:
                closest_ratio = round(price_ratio / 10) * 10  # Round to nearest 10
            else:
                closest_ratio = round(price_ratio / 50) * 50  # Round to nearest 50
            
            is_fractional = True
        
        return {
            "symbol": symbol,
            "actual_price": actual_price,
            "user_paid_price": user_paid_price,
            "calculated_ratio": price_ratio,
            "is_fractional_offering": is_fractional,
            "estimated_fraction": 1 / closest_ratio if is_fractional else 1,
            "estimated_ratio": f"1:{closest_ratio}" if is_fractional else "1:1",
            "actual_shares_owned": 1 / closest_ratio if is_fractional else 1,
            "explanation": f"You own {1/closest_ratio:.6f} shares of the actual stock" if is_fractional else "You own full shares",
            "confidence": "high" if abs(price_ratio - closest_ratio) < (closest_ratio * 0.05) else "medium"
        }
    except Exception as e:
        return {"error": f"Error analyzing fractional offering for {symbol}: {str(e)}"}

In [8]:
@tool
def calculate_fractional_position_value(
    symbol: str, 
    broker_quantity: float, 
    price_paid_per_unit: float
) -> Dict[str, Any]:
    """ Calculate position value accounting for fractional share offerings. Use this tool when you need to calculate the current value and P&L for a stock position, accounting for fractional share offerings. """
    try:
        # Detect if this is a fractional offering
        fractional_info = detect_fractional_share_offering(symbol, price_paid_per_unit)
        if "error" in fractional_info:
            return fractional_info
        
        # Get current market price
        stock_info = get_stock_information(symbol)
        if "error" in stock_info:
            return stock_info
        
        current_actual_price = stock_info["current_price"]
        
        if fractional_info["is_fractional_offering"]:
            # Calculate actual shares owned
            fraction_per_unit = fractional_info["estimated_fraction"]
            actual_shares_owned = broker_quantity * fraction_per_unit
            
            # Calculate values
            current_market_value = actual_shares_owned * current_actual_price
            cost_basis = broker_quantity * price_paid_per_unit
            gain_loss = current_market_value - cost_basis
            gain_loss_percent = (gain_loss / cost_basis) * 100 if cost_basis > 0 else 0
            
            return {
                "symbol": symbol,
                "is_fractional_offering": True,
                "broker_units_owned": broker_quantity,
                "actual_shares_owned": actual_shares_owned,
                "fraction_ratio": fractional_info["estimated_ratio"],
                "current_actual_share_price": current_actual_price,
                "price_paid_per_unit": price_paid_per_unit,
                "current_market_value": current_market_value,
                "cost_basis": cost_basis,
                "gain_loss": gain_loss,
                "gain_loss_percent": gain_loss_percent,
                "explanation": f"Your broker offers 1 unit = {fraction_per_unit:.4f} actual shares"
            }
        else:
            # Regular calculation
            return calculate_position_value(symbol, broker_quantity, price_paid_per_unit, current_actual_price)
            
    except Exception as e:
        return {"error": f"Error calculating fractional position for {symbol}: {str(e)}"}

### Prebuilt tool

In [9]:
from langchain_community.tools import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from tools.youtube_video_transcript import YoutubeVideoTranscriptTool
from tools.image_analyzer import ImageAnalyzer
from tools.document_question_answering_tool import DocumentQuestionAnsweringTool
from tools.youtube_video_transcript import YoutubeVideoTranscriptTool

tavily_web_search = TavilySearchResults(max_results=3)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
youtube_video_transcript = YoutubeVideoTranscriptTool()
image_analyzer = ImageAnalyzer()
document_question_answering = DocumentQuestionAnsweringTool()

In [10]:
available_tools.append(calculate_position_value)
available_tools.append(get_stock_information)
available_tools.append(calculate_position_value_with_splits)
available_tools.append(detect_stock_splits)
available_tools.append(detect_fractional_share_offering)
available_tools.append(calculate_fractional_position_value)
available_tools.append(tavily_web_search)
available_tools.append(wikipedia)
available_tools.append(youtube_video_transcript)
available_tools.append(image_analyzer)
available_tools.append(document_question_answering)
available_tools

[StructuredTool(name='createTransaction', description='Create a new transaction in the portfolio.', args_schema={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Stock ticker symbol'}, 'type': {'description': 'Transaction type (BUY, SELL, DIVIDEND)'}, 'quantity': {'description': 'Quantity of shares'}, 'price': {'description': 'Price per share'}, 'fees': {'description': 'Fees paid per transaction', 'default': '0.00'}, 'isFractional': {'type': 'boolean', 'description': 'Determine if this is an operation on a stock fraction (for fractional offerings)', 'default': False}, 'fractionalMultiplier': {'description': 'Fraction of the real stock option represented by this fractional offered option', 'default': '1.0'}, 'commissionCurrency': {'description': 'Fees currency', 'default': 'USD'}, 'currency': {'description': 'Transaction currency'}, 'date': {'description': 'Transaction date (YYYY-MM-DD)', 'default': 'TODAY'}, 'notes': {'type': 'string', 'description': 'Trans

## Define the main LLM and bind tools

In [11]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.rate_limiters import InMemoryRateLimiter
import os

rate_limiter = InMemoryRateLimiter(requests_per_second=2, check_every_n_seconds=0.2, max_bucket_size=5)

llm = ChatOllama(model="gpt-oss:20b", num_ctx=131072, temperature=0.2)
#llm = ChatAnthropic(model="claude-3-5-haiku-latest", rate_limiter=rate_limiter)
#llm = ChatOpenAI(model="gpt-4.1", rate_limiter=rate_limiter)
llm_with_tools = llm.bind_tools(available_tools)

# Planner node

In [12]:
from langchain_core.messages import SystemMessage

planner_llm = ChatOllama(model="gpt-oss:20b", num_ctx=131072, temperature=0.2)
#planner_llm = ChatAnthropic(model="claude-3-5-haiku-latest")
#planner_llm = ChatOpenAI(model="gpt-4o")

tool_descriptions = []
for tool in available_tools:
    name = tool.name
    desc = tool.description
    tool_descriptions.append(f"- {name}: {desc}")

tools_list_text = "\n".join(tool_descriptions)

def planner(state: AgentState):
    print("\n----- Running planner -----\n")
    original_input = state.get("original_user_input", "")

    system_prompt = (
        f"""
        You are a strategic planning agent for a comprehensive portfolio management system. Your role is to analyze user requests and create detailed, step-by-step execution plans to accomplish their goals. You have knowledge of all available tools but CANNOT execute them directly - you only create plans for other agents to follow.
        Available Tools in the System:
        {tools_list_text}

        ## Your Planning Process
        For each user request, follow this structured approach:

        1. **REQUEST ANALYSIS**: 
        - Identify the user's primary goal
        - Determine required data inputs
        - Identify potential risks or edge cases

        2. **STEP-BY-STEP PLAN**:
        - Break down the request into logical, sequential steps
        - Specify which tool should be used for each step
        - Include necessary parameters and data flow between steps
        - Consider error handling and validation points

        3. **DEPENDENCIES & PREREQUISITES**:
        - Identify what data must be gathered first
        - Note any required validations or checks
        - Specify parallel vs sequential execution requirements

        4. **SUCCESS CRITERIA**:
        - Define what constitutes successful completion
        - Specify expected outputs or outcomes
        - Include verification steps

        ## Planning Guidelines

        - **Be Specific**: Name exact tools and parameters needed
        - **Consider Data Flow**: Ensure outputs from one step feed properly into the next
        - **Include Validation**: Plan for data verification and error checking
        - **Think Holistically**: Consider portfolio impact, not just individual transactions
        - **Plan for Errors**: Include fallback strategies and rollback procedures
        - **Optimize Efficiency**: Suggest parallel execution where appropriate
        """
    )

    user_prompt = f"User request: {original_input}"

    try:
        plan_message = planner_llm.invoke([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ])
    except Exception as e:
        print(f"Error calling to planner LLM: {e}")
        return {
            "original_user_input": original_input,
            "messages": state['messages'],
            "execution_plan": state['execution_plan']
        }

    plan_user_message = HumanMessage(content=f"Execution Plan:\n{plan_message.content}")

    # Combine with existing messages properly
    existing_messages = state.get('messages', [])
    updated_messages = existing_messages + [plan_user_message]

    return {
        "original_user_input": original_input,
        "messages": updated_messages,
        "execution_plan": plan_message
    }

## Assistant node

In [13]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

def sanitize_messages_for_anthropic(messages):
    """
    Sanitize messages for Anthropic API requirements:
    - Only one system message at the beginning
    - Convert additional system messages to user messages
    - Ensure proper message ordering
    """
    if not messages:
        return messages
    
    # Separate system and non-system messages
    system_messages = []
    other_messages = []
    
    for msg in messages:
        if isinstance(msg, SystemMessage):
            system_messages.append(msg)
        else:
            other_messages.append(msg)
    
    # Combine all system content into one system message
    if system_messages:
        combined_system_content = "\n\n".join([msg.content for msg in system_messages])
        # Only keep the first system message with combined content
        sanitized_messages = [SystemMessage(content=combined_system_content)]
        sanitized_messages.extend(other_messages)
    else:
        sanitized_messages = other_messages
    
    return sanitized_messages

print("✅ Anthropic message sanitization helper loaded")


✅ Anthropic message sanitization helper loaded


In [14]:
from langchain_core.messages import SystemMessage

def assistant(state: AgentState):
    print("\n\n\n ----- Running assistant... ----- \n\n\n")
    try:
        sanitized_messages = sanitize_messages_for_anthropic(state['messages'])
        response = llm_with_tools.invoke(sanitized_messages)
    except Exception as e:
        print(f"Error calling to assistant LLM with tools: {e}")
        return {
            "original_user_input": state["original_user_input"],
            "messages": state['messages'],
            "execution_plan": state['execution_plan']
        }

    print(f"State: {response}")
    
    return {
        "original_user_input": state["original_user_input"],
        "messages": state['messages'] + [response],
        "execution_plan": state['execution_plan']
    }

## Build the graph

In [15]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(AgentState)

tools_node = ToolNode(available_tools)
graph_builder.add_node("planner", planner)
graph_builder.add_node("assistant", assistant)
graph_builder.add_node("tools", tools_node)

graph_builder.add_edge(START, "planner")
graph_builder.add_edge("planner", "assistant")
graph_builder.add_conditional_edges("assistant", tools_condition)
graph_builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [43]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=True).draw_mermaid_png(max_retries=5, retry_delay=3.0)))

KeyboardInterrupt: 

## Gradio Integration - Portfolio Assistant Chatbot


In [16]:
## Enhanced Debug Functions with Context Monitoring

async def get_context_analysis_async(thread_id_input=None):
    """Analyze context usage for a specific thread (async version)"""
    try:
        debug_thread_id = thread_id_input.strip() if thread_id_input else current_thread_id
        
        if not debug_thread_id:
            return "❌ No thread ID available. Start a conversation first or provide a thread ID."
        
        # Get state from graph memory
        config = {"configurable": {"thread_id": debug_thread_id}}
        
        try:
            state = await graph.aget_state(config)
            
            if not state or not state.values:
                return f"📭 No state found for thread ID: {debug_thread_id}"
            
            messages = state.values.get("messages", [])
            
            # Generate context analysis report
            report = context_monitor.get_context_summary_report(messages, SYSTEM_PROMPT)
            
            return f"🔍 CONTEXT ANALYSIS FOR THREAD: {debug_thread_id}\n{report}"
            
        except Exception as e:
            return f"❌ Error accessing graph state: {str(e)}"
            
    except Exception as e:
        return f"❌ Context analysis error: {str(e)}"

def get_context_analysis(thread_id_input=None):
    """Sync wrapper for the async context analysis function"""
    try:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        try:
            return loop.run_until_complete(get_context_analysis_async(thread_id_input))
        finally:
            loop.close()
    except Exception as e:
        return f"❌ Context analysis error: {str(e)}"

# Function to update context monitor model if you switch LLMs
def update_context_monitor_model(new_model_name: str):
    """Update the context monitor when switching models"""
    global context_monitor
    context_monitor = ContextMonitor(new_model_name)
    print(f"📊 Context monitor updated to: {new_model_name}")
    print(f"📏 New context limit: {context_monitor.context_limits.get(new_model_name, 'Unknown'):,} tokens")

print("✅ Enhanced debug functions with context monitoring loaded!")


✅ Enhanced debug functions with context monitoring loaded!


In [17]:
## Context Size Monitoring System

import tiktoken
from typing import Dict, Any, List, Optional, Union
from langchain_core.messages import BaseMessage
import json

class ContextMonitor:
    """Monitor and track context size across different LLM providers"""
    
    def __init__(self, model_name: str = "gpt-oss:20b"):
        self.model_name = model_name
        self.context_limits = {
            # OpenAI models
            "gpt-4": 8192,
            "gpt-4-32k": 32768,
            "gpt-4-turbo": 128000,
            "gpt-4o": 128000,
            # Anthropic models
            "claude-3-sonnet": 200000,
            "claude-3-opus": 200000,
            "claude-3-haiku": 200000,
            "claude-3-5-sonnet": 200000,
            "claude-3-5-haiku": 200000,
            # Ollama models (approximate)
            "gpt-oss:20b": 131072,  # Based on your num_ctx setting
            "llama2": 4096,
            "llama3": 8192,
            "mistral": 32768,
            # Default fallback
            "default": 4096
        }
        
        # Initialize tokenizer based on model
        self.tokenizer = self._get_tokenizer()
        
    def _get_tokenizer(self):
        """Get appropriate tokenizer for the model"""
        try:
            if "gpt" in self.model_name.lower() and not "gpt-oss" in self.model_name.lower():
                # OpenAI models
                return tiktoken.encoding_for_model("gpt-4")
            elif "claude" in self.model_name.lower():
                # For Anthropic, we'll use a rough approximation
                return tiktoken.get_encoding("cl100k_base")
            else:
                # Fallback for other models (Ollama, etc.)
                return tiktoken.get_encoding("cl100k_base")
        except Exception as e:
            print(f"Warning: Could not load tokenizer for {self.model_name}, using fallback: {e}")
            return tiktoken.get_encoding("cl100k_base")
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in a text string"""
        try:
            if isinstance(text, str):
                return len(self.tokenizer.encode(text))
            return 0
        except Exception as e:
            # Fallback: rough approximation (4 chars = 1 token)
            print(f"Token counting failed, using approximation: {e}")
            return len(str(text)) // 4
    
    def count_message_tokens(self, message: BaseMessage) -> int:
        """Count tokens in a single message"""
        total_tokens = 0
        
        # Count content tokens
        if hasattr(message, 'content') and message.content:
            total_tokens += self.count_tokens(str(message.content))
        
        # Count tool call tokens
        if hasattr(message, 'tool_calls') and message.tool_calls:
            for tool_call in message.tool_calls:
                total_tokens += self.count_tokens(json.dumps(tool_call))
        
        # Count additional kwargs
        if hasattr(message, 'additional_kwargs') and message.additional_kwargs:
            total_tokens += self.count_tokens(json.dumps(message.additional_kwargs))
        
        # Add message overhead (role, metadata, etc.)
        total_tokens += 10  # Approximate overhead per message
        
        return total_tokens
    
    def analyze_conversation_context(self, messages: List[BaseMessage], system_prompt: str = "") -> Dict[str, Any]:
        """Analyze the full conversation context"""
        
        # Count system prompt tokens
        system_tokens = self.count_tokens(system_prompt) if system_prompt else 0
        
        # Count message tokens
        message_tokens = sum(self.count_message_tokens(msg) for msg in messages)
        
        # Total context
        total_tokens = system_tokens + message_tokens
        
        # Get context limit for current model
        context_limit = self.context_limits.get(self.model_name, self.context_limits["default"])
        
        # Calculate usage percentages
        usage_percentage = (total_tokens / context_limit) * 100
        
        # Determine status
        if usage_percentage < 60:
            status = "🟢 SAFE"
        elif usage_percentage < 80:
            status = "🟡 CAUTION"
        elif usage_percentage < 95:
            status = "🟠 WARNING" 
        else:
            status = "🔴 CRITICAL"
        
        return {
            "total_tokens": total_tokens,
            "system_tokens": system_tokens,
            "message_tokens": message_tokens,
            "context_limit": context_limit,
            "usage_percentage": round(usage_percentage, 2),
            "remaining_tokens": context_limit - total_tokens,
            "status": status,
            "model": self.model_name,
            "message_count": len(messages),
            "breakdown": {
                "system_prompt": system_tokens,
                "conversation": message_tokens,
                "available": context_limit - total_tokens
            }
        }
    
    def should_truncate_context(self, messages: List[BaseMessage], system_prompt: str = "", threshold: float = 0.85) -> bool:
        """Check if context should be truncated"""
        analysis = self.analyze_conversation_context(messages, system_prompt)
        return analysis["usage_percentage"] >= (threshold * 100)
    
    def get_context_summary_report(self, messages: List[BaseMessage], system_prompt: str = "") -> str:
        """Generate a formatted context summary report"""
        analysis = self.analyze_conversation_context(messages, system_prompt)
        
        report = f"""
📊 CONTEXT USAGE REPORT
═══════════════════════════════════════════════════════════════

🤖 Model: {analysis['model']}
📈 Status: {analysis['status']} ({analysis['usage_percentage']}% used)

💾 Token Usage:
   Total Tokens: {analysis['total_tokens']:,}
   Context Limit: {analysis['context_limit']:,}
   Remaining: {analysis['remaining_tokens']:,}

📝 Breakdown:
   System Prompt: {analysis['system_tokens']:,} tokens
   Messages ({analysis['message_count']}): {analysis['message_tokens']:,} tokens

⚠️ Recommendations:
"""
        
        if analysis["usage_percentage"] >= 95:
            report += "   🔴 IMMEDIATE ACTION REQUIRED - Context nearly full!\n"
            report += "   → Consider truncating old messages or starting new conversation\n"
        elif analysis["usage_percentage"] >= 80:
            report += "   🟠 HIGH USAGE - Monitor closely\n"  
            report += "   → Plan context management strategy\n"
        elif analysis["usage_percentage"] >= 60:
            report += "   🟡 MODERATE USAGE - All good for now\n"
            report += "   → Continue monitoring\n"
        else:
            report += "   🟢 LOW USAGE - Plenty of context available\n"
        
        report += "\n═══════════════════════════════════════════════════════════════"
        
        return report

# Initialize global context monitor
context_monitor = ContextMonitor("gpt-oss:20b")  # Match your current model

print("✅ Context monitoring system loaded!")
print(f"📊 Monitoring model: {context_monitor.model_name}")
print(f"📏 Context limit: {context_monitor.context_limits.get(context_monitor.model_name, 'Unknown'):,} tokens")


✅ Context monitoring system loaded!
📊 Monitoring model: gpt-oss:20b
📏 Context limit: 131,072 tokens


In [18]:
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import uuid
import asyncio
from datetime import datetime

# System prompt for the Portfolio Assistant
SYSTEM_PROMPT = """
You are a Portfolio Assistant AI specializing in investment portfolio analysis and management. Your primary role is to provide accurate, insightful analysis while detecting and explaining anomalies that could affect portfolio calculations.

## 🧠 Core Behavior Guidelines

**Always Investigate Anomalies First**: When you see significant price discrepancies (>50% difference between user's cost basis and current market price), immediately investigate potential causes:
- Stock splits: Check if the user's high cost basis suggests missing split adjustments
- Fractional offerings: Check if the user's low cost basis suggests broker fractional products
- Market price changes: Check if the user's cost basis is significantly different from the current market price and try to explain why

**Provide Context with Analysis**: Don't just give numbers—explain what they mean, identify trends, and suggest actions when appropriate.

**Combine Tools Strategically**: Use multiple tools together to provide comprehensive insights rather than isolated data points.

## 🎯 Common Analysis Scenarios

**Scenario 1: Price Discrepancy Investigation**
- User: "I bought TSLA at $15 per share but it's trading at $250"
- Workflow: detect_stock_splits → calculate_position_value_with_splits → explain split history impact
- Always explain how splits affected their actual cost basis and current position value

**Scenario 2: Fractional Share Products**
- User: "I paid $50 for 1 share of Berkshire Hathaway" (BRK.A trades at $400k+)
- Workflow: detect_fractional_share_offering → calculate_fractional_position_value → explain broker's fractional structure
- Clarify how many actual shares they own and what their real exposure is

**Scenario 3: Comprehensive Stock Analysis**
- User: "Tell me about my Apple position and the stock outlook"
- Workflow: getPositionByTicker → get_stock_information → TavilySearchResults (recent news) → WikipediaQueryRun (company background)
- Combine portfolio performance with market context and company fundamentals

**Scenario 4: Document Processing**
- User uploads broker statement or transaction record
- Workflow: ImageAnalyzer or DocumentQuestionAnsweringTool → extract transaction data → verify/update portfolio using transaction tools
- Always cross-reference extracted data with existing portfolio records

**Scenario 5: Portfolio Health Check**
- User: "How is my portfolio performing?"
- Workflow: getPortfolioSummary → getAllPositions → identify outliers → investigate anomalies → get current market data → provide performance context
- Look for positions with unusual P&L ratios that might indicate data issues

**Scenario 6: Adding New Stock Purchase**
- User: "Add a buy transaction: 100 shares of NVDA at $850 on 2024-01-15"
- Workflow: createTransaction → get_stock_information → updateMarketData (with current price) → recalculatePosition → provide transaction confirmation with current market context
- Always refresh market data after adding transactions to ensure accurate position valuations

## ⚡ Decision-Making Rules

1. **Start Broad, Then Focus**: Begin with portfolio overview, then drill into specific issues
2. **Question Unusual Data**: If something looks too good/bad to be true, investigate before reporting
3. **Update Stale Data**: If market prices seem outdated, refresh them before analysis  
4. **Recalculate After Changes**: Always recalculate positions after making transaction modifications
5. **Provide Actionable Insights**: Don't just report data—suggest what the user should consider doing

## 🔍 Red Flags to Investigate

- Cost basis dramatically different from current market price (>100% variance)
- Positions showing extreme gains/losses without clear market justification
- Missing or zero market values for active positions
- Transaction dates that don't align with reported position metrics

Remember: Accuracy is paramount. It's better to take an extra step to verify data than provide misleading analysis.
"""

# Global variable to store the current thread_id for debugging
current_thread_id = None

def format_message_for_debug(msg, index):
    """Format a single message for readable display"""
    msg_type = type(msg).__name__
    timestamp = datetime.now().strftime("%H:%M:%S")
    
    if hasattr(msg, 'content'):
        content = msg.content
    else:
        content = str(msg)
    
    # Handle tool calls if present
    tool_info = ""
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        tool_info = f"\n   🔧 Tool Calls: {[tc.get('name', 'Unknown') for tc in msg.tool_calls]}"
    
    return f"""
┌─ Message {index + 1}: {msg_type} [{timestamp}]
│  📝 Content: {content}
│  🏷️  Additional: {getattr(msg, 'additional_kwargs', {})}{tool_info}
└─ """

async def get_graph_memory_debug_async(thread_id_input=None):
    """Retrieve and format all messages from graph memory for debugging (async version)"""
    try:
        # Use provided thread_id or current one
        debug_thread_id = thread_id_input.strip() if thread_id_input else current_thread_id
        
        if not debug_thread_id:
            return "❌ No thread ID available. Start a conversation first or provide a thread ID."
        
        # Get state from graph memory
        config = {"configurable": {"thread_id": debug_thread_id}}
        
        try:
            state = await graph.aget_state(config)  # Use async version
            
            if not state or not state.values:
                return f"📭 No state found for thread ID: {debug_thread_id}"
            
            messages = state.values.get("messages", [])
            original_input = state.values.get("original_user_input", "N/A")
            execution_plan = state.values.get("execution_plan", "N/A")
            
            # Format output
            debug_output = f"""
🔍 LANGGRAPH MEMORY DEBUG
═══════════════════════════════════════════════════════════════

🆔 Thread ID: {debug_thread_id}
📋 Original Input: {original_input}
💬 Total Messages: {len(messages)}
📝 Execution Plan: {type(execution_plan).__name__ if execution_plan else "None"}

📨 MESSAGE HISTORY:
═══════════════════════════════════════════════════════════════
"""
            
            if messages:
                for i, msg in enumerate(messages):
                    debug_output += format_message_for_debug(msg, i)
            else:
                debug_output += "\n📭 No messages found in memory."
            
            debug_output += f"""
═══════════════════════════════════════════════════════════════

🔧 GRAPH STATE SUMMARY:
- State Keys: {list(state.values.keys()) if state.values else "None"}
- Next Node: {state.next or "END"}
- Config: {state.config}

🕒 Last Updated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
═══════════════════════════════════════════════════════════════
"""
            
            return debug_output
            
        except Exception as e:
            return f"❌ Error accessing graph state: {str(e)}"
            
    except Exception as e:
        return f"❌ Debug error: {str(e)}"

def get_graph_memory_debug(thread_id_input=None):
    """Sync wrapper for the async debug function"""
    try:
        # Create new event loop for this sync function
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        try:
            return loop.run_until_complete(get_graph_memory_debug_async(thread_id_input))
        finally:
            loop.close()
    except Exception as e:
        return f"❌ Debug error: {str(e)}"

def reset_conversation():
    """Reset the conversation thread and clear chat UI"""
    global current_thread_id
    old_thread_id = current_thread_id
    current_thread_id = str(uuid.uuid4())
    
    status_message = f"""🔄 Conversation Reset Complete!

📋 Details:
• Previous Thread ID: {old_thread_id or 'None'}
• New Thread ID: {current_thread_id}
• Chat history cleared
• Fresh conversation started

💡 You can now start a new conversation with a clean slate!"""
    
    return status_message, None  # Return status and None to clear chat

async def chat_with_portfolio_assistant(message, history):
    """
    Handle chat interaction with the Portfolio Assistant (SIMPLIFIED VERSION)
    """
    global current_thread_id
    
    try:
        # Use persistent thread ID for the conversation
        if current_thread_id is None:
            current_thread_id = str(uuid.uuid4())
        
        config = {"configurable": {"thread_id": current_thread_id}}
        
        messages = [
            SystemMessage(content=SYSTEM_PROMPT),
            HumanMessage(content=message)
        ]
        
        # Prepare initial state
        initial_state = {
            "messages": messages,
            "original_user_input": message,
            "execution_plan": ""
        }
        
        # Use ASYNC invocation for MCP tools compatibility
        print(f"🔄 Processing request: {message[:50]}...")
        print(f"🆔 Thread ID: {current_thread_id}")
        
        # Use ainvoke for async tools
        final_state = await graph.ainvoke(initial_state, config=config)
        
        # Extract the assistant's response
        response_content = ""
        if final_state.get("messages"):
            # Find the last AI message
            for msg in reversed(final_state["messages"]):
                if hasattr(msg, 'content') and isinstance(msg, AIMessage):
                    response_content = clean_llm_response(msg.content)
                    break
        
        if not response_content:
            return "I apologize, but I encountered an issue processing your request. Please try again."
        
        print(f"✅ Response ready ({len(response_content)} chars)")
        return response_content
        
    except Exception as e:
        print(f"❌ Error in chat function: {e}")
        import traceback
        traceback.print_exc()
        return f"I encountered an error: {str(e)}. Please try your request again."

import re

def clean_llm_response(response: str) -> str:
    """Remove thinking sections that break Gradio display"""
    
    # Remove common thinking section patterns
    patterns = [
        r'<thinking>.*?</thinking>',
        r'<think>.*?</think>',
        r'<thought>.*?</thought>', 
        r'<reasoning>.*?</reasoning>',
        r'\[thinking\].*?\[/thinking\]',
        r'<!--.*?thinking.*?-->'
    ]
    
    cleaned = response
    for pattern in patterns:
        cleaned = re.sub(pattern, '', cleaned, flags=re.DOTALL | re.IGNORECASE)
    
    # Clean up extra whitespace
    cleaned = re.sub(r'\n\s*\n\s*\n', '\n\n', cleaned)
    cleaned = cleaned.strip()
    
    # Return fallback if empty
    return cleaned if cleaned else "Response processed successfully."

# Create Gradio interface
def create_portfolio_chatbot():
    """
    Create and configure the Gradio chatbot interface with debug functionality
    """
    
    with gr.Blocks(
        title="Portfolio Assistant", 
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            max-width: 1400px !important;
        }
        .chat-message {
            border-radius: 10px !important;
        }
        .debug-section {
            border: 2px solid #e1e5e9;
            border-radius: 8px;
            padding: 10px;
            margin: 10px 0;
        }
        """
    ) as interface:
        
        gr.Markdown(
            """
            # 📈 Portfolio Assistant
            
            Welcome to your AI-powered Portfolio Assistant! I can help you manage and analyze your investment portfolio using advanced tools and real-time data.
            
            **What I can help you with:**
            - 📊 Portfolio analysis and performance metrics
            - 💼 Transaction management (create, update, delete)
            - 💰 Position tracking and market data updates
            - 🔍 Advanced search and filtering
            - 📈 Investment insights and recommendations
            
            **💬 Chat with your Portfolio Assistant below:**
            Ask me anything about your portfolio - I have access to all your investment data!
            """
        )
        
        # Main chat interface
        chatbot_interface = gr.ChatInterface(
            fn=chat_with_portfolio_assistant,
            examples=[
                "What's my current portfolio summary?",
                "Show me all my transactions for AAPL",
                "Add a buy transaction: 50 shares of MSFT at $350 on 2024-01-15",
                "What are my performance metrics?",
                "Update Tesla's current price to $245",
                "What's my position in Apple stock?",
                "Search for all transactions in the last 30 days"
            ],
            type="messages"
        )
        
        # Debug section
        gr.Markdown("---")
        gr.Markdown("## 🔍 Debug & Control Tools", elem_classes=["debug-section"])
        
        with gr.Row():
            with gr.Column(scale=2):
                thread_id_input = gr.Textbox(
                    label="🆔 Thread ID (optional)",
                    placeholder="Leave empty to use current conversation thread",
                    info="Enter a specific thread ID to debug, or leave empty to debug current conversation"
                )
            
            with gr.Column(scale=1):
                with gr.Row():
                    debug_btn = gr.Button("🔍 View Memory", variant="secondary")
                    context_btn = gr.Button("📊 Context Analysis", variant="secondary")
                    reset_btn = gr.Button("🔄 Reset Conversation", variant="stop")
        
        # Status output for reset
        reset_status = gr.Textbox(
            label="🔄 Reset Status",
            placeholder="Click 'Reset Conversation' to start fresh...",
            lines=6,
            show_copy_button=False,
            elem_classes=["debug-section"]
        )
        
        # Debug output area
        debug_output = gr.Textbox(
            label="📋 Graph Memory Debug Output",
            placeholder="Click 'View Memory' to see the current graph state and message history...",
            lines=20,
            max_lines=30,
            show_copy_button=True,
            elem_classes=["debug-section"]
        )
        
        # Context analysis output area
        context_output = gr.Textbox(
            label="📊 Context Usage Analysis",
            placeholder="Click 'Context Analysis' to see token usage and context health...",
            lines=15,
            max_lines=20,
            show_copy_button=True,
            elem_classes=["debug-section"]
        )
        
        # Current thread ID display
        current_thread_display = gr.Textbox(
            label="🆔 Current Thread ID",
            value=lambda: current_thread_id or "No active conversation",
            interactive=False
        )
        
        # Wire up the functionality
        debug_btn.click(
            fn=get_graph_memory_debug,
            inputs=[thread_id_input],
            outputs=[debug_output]
        )
        
        context_btn.click(
            fn=get_context_analysis,
            inputs=[thread_id_input],
            outputs=[context_output]
        )
        
        # ✅ FIXED: Reset conversation properly
        reset_btn.click(
            fn=reset_conversation,
            inputs=[],
            outputs=[reset_status, chatbot_interface.chatbot]  # Clear both status and chat
        ).then(
            fn=lambda: current_thread_id or "No active conversation",
            inputs=[],
            outputs=[current_thread_display]  # Update thread ID display
        )
        
        # Add footer information
        gr.Markdown(
            """
            ---
            
            **💡 Tips:**
            - Be specific when adding transactions (include ticker, quantity, price, date)
            - Ask for summaries to get quick overviews
            - Use ticker symbols (e.g., AAPL, MSFT, TSLA) for best results
            - I can handle multiple requests in one message
            
            **🔧 Tools Available:** Transaction Management, Portfolio Analysis, Position Tracking, Performance Metrics, Market Data Updates
            
            **🔍 Debug & Control Tools:**
            - **View Memory**: See all messages and state in the current conversation thread
            - **Context Analysis**: Monitor token usage and detect when context is getting full
            - **Reset Conversation**: Clear chat history and start with a new thread ID
            - **Thread ID**: Each conversation gets a unique ID for memory isolation
            
            **📊 Context Monitoring:**
            - 🟢 SAFE (< 60%): Plenty of context available
            - 🟡 CAUTION (60-80%): Moderate usage, monitor closely  
            - 🟠 WARNING (80-95%): High usage, plan context management
            - 🔴 CRITICAL (> 95%): Risk of hallucination, start new conversation
            """
        )
    
    return interface

# Create and launch the chatbot
print("🚀 Setting up Portfolio Assistant Chatbot with Reset Functionality...")
portfolio_chatbot = create_portfolio_chatbot()

print("✅ Portfolio Assistant is ready!")
print("📝 The chatbot interface has been created and is ready to launch.")
print("💡 Features included:")
print("   - 💬 Full conversation interface")
print("   - 🔍 Memory debugging tools")
print("   - 🆔 Thread ID tracking")
print("   - 📋 Message history viewer")
print("   - ⚡ Async tool support for MCP")
print("   - 🔄 Working reset conversation functionality")
print("💡 Run portfolio_chatbot.launch() to start the interface!")

🚀 Setting up Portfolio Assistant Chatbot with Reset Functionality...
✅ Portfolio Assistant is ready!
📝 The chatbot interface has been created and is ready to launch.
💡 Features included:
   - 💬 Full conversation interface
   - 🔍 Memory debugging tools
   - 🆔 Thread ID tracking
   - 📋 Message history viewer
   - ⚡ Async tool support for MCP
   - 🔄 Working reset conversation functionality
💡 Run portfolio_chatbot.launch() to start the interface!


In [19]:
# Launch the Portfolio Assistant Chatbot
if __name__ == "__main__":
    print("🎯 Starting Portfolio Assistant Chatbot...")
    print("🔗 Make sure your MCP server is running on localhost:8081")
    print("📡 Launching Gradio interface...")
    
    # Launch with share=False for local use, set share=True to create public link
    portfolio_chatbot.launch(
        server_name="127.0.0.1",  # Local access only
        server_port=7860,         # Default Gradio port
        share=False,              # Set to True for public sharing
        debug=True,               # Enable debug mode
        show_error=True,          # Show detailed error messages
        quiet=False,              # Show startup logs
        inbrowser=True,           # Auto-open in browser
        height=800,               # Interface height
        favicon_path=None,        # You can add a custom favicon
        auth=None,                # Add authentication if needed: auth=("username", "password")
    )
else:
    print("📝 To launch the chatbot, run: portfolio_chatbot.launch()")
    print("💡 Example: portfolio_chatbot.launch(share=True) for public access")


🎯 Starting Portfolio Assistant Chatbot...
🔗 Make sure your MCP server is running on localhost:8081
📡 Launching Gradio interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


🔄 Processing request: What's my current portfolio summary?...
🆔 Thread ID: 9f64cd4d-bd10-4876-87bf-81e762a9479a

----- Running planner -----




 ----- Running assistant... ----- 



State: content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-24T16:20:00.513251Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2843693292, 'load_duration': 64588625, 'prompt_eval_count': 2376, 'prompt_eval_duration': 2173987125, 'eval_count': 39, 'eval_duration': 598976584, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='getPortfolioSummary', arguments={}))])} id='run--50177e5e-4788-46c7-acfe-b84817bffc42-0' tool_calls=[{'name': 'getPortfolioSummary', 'args': {}, 'id': 'fcfd5510-333d-48f9-b5d0-dcf4de6154d9', 'type': 'tool_call'}] usage_metadata={'input_tokens': 2376, 'output_tokens': 39, 'total_tokens': 2415}



 ----- Running assistant... ----- 



State: content='**Portfolio Snapshot (as of t

$BATS: possibly delisted; no price data found  (period=1d)





 ----- Running assistant... ----- 



State: content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-24T16:42:43.891199Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10771733875, 'load_duration': 103658584, 'prompt_eval_count': 20543, 'prompt_eval_duration': 586971250, 'eval_count': 77, 'eval_duration': 3332164083, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_stock_information', arguments={'symbol': 'BATS.L'}))])} id='run--a805249d-405b-4595-8dcf-99cded3244e1-0' tool_calls=[{'name': 'get_stock_information', 'args': {'symbol': 'BATS.L'}, 'id': 'c745686f-0153-4d38-90a7-a1b1e307a739', 'type': 'tool_call'}] usage_metadata={'input_tokens': 20543, 'output_tokens': 77, 'total_tokens': 20620}



 ----- Running assistant... ----- 



State: content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-24T16:42:53.379156Z', 'done': True, '

$BATS: possibly delisted; no price data found  (period=1d)


...




 ----- Running assistant... ----- 



State: content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-24T16:44:48.203013Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15666291375, 'load_duration': 100204041, 'prompt_eval_count': 21263, 'prompt_eval_duration': 741666875, 'eval_count': 21, 'eval_duration': 899839459, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_stock_information', arguments={'symbol': 'BATS.L'}))])} id='run--4442b800-c38a-4155-b8d1-5afa755c3a1f-0' tool_calls=[{'name': 'get_stock_information', 'args': {'symbol': 'BATS.L'}, 'id': '52525a4a-eb17-448c-9f78-3586682a7196', 'type': 'tool_call'}] usage_metadata={'input_tokens': 21263, 'output_tokens': 21, 'total_tokens': 21284}
❌ Error in chat function: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, v

Traceback (most recent call last):
  File "/var/folders/t8/ts_pzf9x5ll1xbm9_7jrxstr0000gn/T/ipykernel_66800/4040045585.py", line 219, in chat_with_portfolio_assistant
    final_state = await graph.ainvoke(initial_state, config=config)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pacama95/venv/lib/python3.13/site-packages/langgraph/pregel/__init__.py", line 2794, in ainvoke
    async for chunk in self.astream(
    ...<13 lines>...
            chunks.append(chunk)
  File "/Users/pacama95/venv/lib/python3.13/site-packages/langgraph/pregel/__init__.py", line 2698, in astream
    raise GraphRecursionError(msg)
langgraph.errors.GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT


🔄 Processing request: My BATS position is wrong, I bought it in London S...
🆔 Thread ID: 23505eeb-dadd-41fc-98d3-c297936d1107

----- Running planner -----




 ----- Running assistant... ----- 



State: content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-24T16:46:22.427805Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3910617708, 'load_duration': 59148333, 'prompt_eval_count': 3394, 'prompt_eval_duration': 3060411250, 'eval_count': 46, 'eval_duration': 782688917, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='searchTransactions', arguments={'ticker': 'BATS'}))])} id='run--8b4dd9d6-82d4-4e96-919c-3905c5a802eb-0' tool_calls=[{'name': 'searchTransactions', 'args': {'ticker': 'BATS'}, 'id': 'c2d600e2-4ed5-40fc-91be-93ecf655d042', 'type': 'tool_call'}] usage_metadata={'input_tokens': 3394, 'output_tokens': 46, 'total_tokens': 3440}



 ----- Running assistant... ----- 



S

/var/folders/t8/ts_pzf9x5ll1xbm9_7jrxstr0000gn/T/ipykernel_66800/1515556602.py:6: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  stock_info = get_stock_information(symbol)





 ----- Running assistant... ----- 



State: content='' additional_kwargs={} response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-08-24T16:48:27.136169Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1812864834, 'load_duration': 101683125, 'prompt_eval_count': 7115, 'prompt_eval_duration': 509506792, 'eval_count': 20, 'eval_duration': 438763000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='searchTransactions', arguments={'ticker': 'BATS.L'}))])} id='run--b38b33a8-6700-4a6c-b9d7-c1855f805bea-0' tool_calls=[{'name': 'searchTransactions', 'args': {'ticker': 'BATS.L'}, 'id': '1c0e81e7-efec-49e7-8614-f0a1e3c7fd00', 'type': 'tool_call'}] usage_metadata={'input_tokens': 7115, 'output_tokens': 20, 'total_tokens': 7135}



 ----- Running assistant... ----- 



State: content='### Quick snapshot of your **BATS.L** position\n\n| Metric | Value |\n|--------|-------|\n| Shares recorded | **170.0** |\n| Avg cost (pe

## Tests section

In [27]:
# Test the Context Monitoring System
def test_context_monitoring():
    """Test the context monitoring with sample messages"""
    from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
    
    # Create sample messages to test
    sample_messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content="What's my portfolio summary?"),
        AIMessage(content="Here's your portfolio summary with detailed breakdown..."),
        HumanMessage(content="Can you analyze my Apple position?"),
        AIMessage(content="Let me analyze your AAPL position in detail...")
    ]
    
    print("🧪 Testing Context Monitoring System...")
    print("=" * 60)
    
    # Test basic token counting
    test_text = "This is a sample message for testing token counting."
    token_count = context_monitor.count_tokens(test_text)
    print(f"✅ Token counting test: '{test_text}' = {token_count} tokens")
    
    # Test conversation analysis
    analysis = context_monitor.analyze_conversation_context(sample_messages, SYSTEM_PROMPT)
    print(f"\n✅ Conversation Analysis:")
    print(f"   Total tokens: {analysis['total_tokens']:,}")
    print(f"   Usage: {analysis['usage_percentage']}% {analysis['status']}")
    print(f"   Remaining: {analysis['remaining_tokens']:,} tokens")
    
    # Test full report
    print(f"\n✅ Full Context Report:")
    report = context_monitor.get_context_summary_report(sample_messages, SYSTEM_PROMPT)
    print(report)
    
    print("\n🎉 Context monitoring system is working correctly!")
    return True

# Run the test
test_context_monitoring()


🧪 Testing Context Monitoring System...
✅ Token counting test: 'This is a sample message for testing token counting.' = 10 tokens

✅ Conversation Analysis:
   Total tokens: 872
   Usage: 0.67% 🟢 SAFE
   Remaining: 130,200 tokens

✅ Full Context Report:

📊 CONTEXT USAGE REPORT
═══════════════════════════════════════════════════════════════

🤖 Model: gpt-oss:20b
📈 Status: 🟢 SAFE (0.67% used)

💾 Token Usage:
   Total Tokens: 872
   Context Limit: 131,072
   Remaining: 130,200

📝 Breakdown:
   System Prompt: 784 tokens
   Messages (5): 88 tokens

⚠️ Recommendations:
   🟢 LOW USAGE - Plenty of context available

═══════════════════════════════════════════════════════════════

🎉 Context monitoring system is working correctly!


True